<a href="https://colab.research.google.com/github/jesvin1/Basics/blob/master/GARCH_ARCH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>